<a href="https://colab.research.google.com/github/rbqpark/tinger/blob/main/KNN_BPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**KNN BPM Only Experiment**

### Import libraries and other dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

### Import data to use as inputs for full KNN model. Clean the data for missing values.

In [ ]:
mandarin = gc.open('Playlist Data (Mandarin)').sheet1
# get_all_values gives a list of rows.
playlist_mandarin = mandarin.get_all_values()
print(playlist_mandarin)

# Convert to a DataFrame and render.
mandarin = pd.DataFrame.from_records(playlist_mandarin)
mandarin.columns = ['Song Title', 'Artist', 'Male0 / Female1/Other3', 'Publish Year', 'Length of Song', 'Length in Seconds', 'Solo0 / Group1', 'Main Chord Progression', 'Romance', 'Lyrics', 'Translated Lyrics ', 'bpm', 'danceability']
mandarin = mandarin.iloc[1:]

[['Song Title', 'Artist', 'Male0 / Female1/Other3', 'Publish Year', 'Length of Song', 'Length in Seconds', 'Solo0 / Group1', 'Main Chord Progression', 'Romance', 'Lyrics', 'Translated Lyrics ', 'bpm', 'danceability'], ['我很快乐', '周兴哲', '0', '2020', '5:11', '311', '0', '1,5,6,3,4,5,3,6', '0', '雷雨依舊 奔跑著 世界像你 模糊停格 總以為能 永遠愛著 時間 卻幫我們上了一課 路 無法倒退 我 錯過機會 我說著 我很快樂 我無所謂 你最懂我的 為你付出 我不後悔 只要你過得好 我不一定 需要擁有 你說要自由 我沒理由 不讓你走 說真的 我很快樂 為你流淚 你找到你的 幸福快樂 就不浪費 要犯過多少錯 終於懂得 愛是什麼 遺憾有多好 愛過的人忘不掉 越努力愛 一個人 越想割捨 越捨不得 總以為能 失而復得 你卻 有更好的未來等著 我 一定成全 我 微笑告別 說真的 我很快樂 為你流淚 你找到你的 幸福快樂 就不浪費 要犯過多少錯 終於懂得 愛是什麼 遺憾有多好 愛過的人忘不掉 我說著 我很快樂 我無所謂 你最懂我的 為你付出 我不後悔 只要你過得好 我不一定 需要擁有 你說要自由 我沒理由 不讓你走 說真的 我很快樂 為你流淚 你找到你的 幸福快樂 就不浪費 多幸運能遇到 最懷念的 你的擁抱 遺憾有多好 愛過的人忘不掉', "Rain still run the world like you always think stop-motion blur can always love the time they help us a lesson way back I could not miss the opportunity I said I'm happy I do not care you know me pay for you as long as I do not regret how are you doing I do not necessarily need to be f

In [ ]:
mandarin_bpm = mandarin.iloc[:,[0,11,12]]
mandarin_bpm

,Song Title,bpm,danceability
1,我很快乐,74.40203094,1.065184236
2,过,128.0996704,1.161730528
3,因为你所以我,,
4,很久以后,145.1845398,1.059653521
5,莲,109.9270706,1.125244498
...,...,...,...
249,水调歌头,129.5627747,1.07304275
250,淒美地,125.0384827,1.560384631
251,你是如此难以忘记,116.7671356,0.881413877
252,致姍姍來遲的你,120.0031357,1.321027279


In [ ]:
mandarin_bpm['bpm'].replace('', np.nan, inplace=True)
mandarin_bpm_clean = mandarin_bpm.dropna(subset = ["bpm"])
mandarin_bpm_clean

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,Song Title,bpm,danceability
1,我很快乐,74.40203094,1.065184236
2,过,128.0996704,1.161730528
4,很久以后,145.1845398,1.059653521
5,莲,109.9270706,1.125244498
6,好想爱这个世界啊,162.0032043,0.9613984227
...,...,...,...
249,水调歌头,129.5627747,1.07304275
250,淒美地,125.0384827,1.560384631
251,你是如此难以忘记,116.7671356,0.881413877
252,致姍姍來遲的你,120.0031357,1.321027279


In [ ]:
man_X = mandarin_bpm_clean.iloc[:, 1:3].values
man_y = mandarin_bpm_clean.iloc[:, 0].values

In [ ]:
english = gc.open('Playlist Data (English)').sheet1
# get_all_values gives a list of rows.
playlist_english = english.get_all_values()
print(playlist_english)

# Convert to a DataFrame and render.
english = pd.DataFrame.from_records(playlist_english)
english.columns = ['Song Title', 'Artist', 'Male0 / Female1/Other3', 'Publish Year', 'Length of Song', 'Length in Seconds', 'Solo0 / Group1', 'Main Chord Progression', 'Romance', 'Lyrics', 'bpm', 'danceability']
english = english.iloc[1:]
english

[['Song Title', 'Artist', 'Male0 / Female1/Other3', 'Publish Year', 'Solo0 / Group1', 'Main Chord Progression', 'Romance', 'Lyrics', 'Length of Song', 'Length in Seconds', 'bpm', 'danceability'], ['We Are Young', 'Fun', '3', '2012', '1', '1,6,4,5,1,6,4,5 ', '0', "Give me a second I, I need to get my story straight My friends are in the bathroom getting higher than the Empire State My lover she's waiting for me just across the bar My seat's been taken by some sunglasses asking bout a scar, and I know I gave it to you months ago I know you're trying to forget But between the drinks and subtle things The holes in my apologies, you know I'm trying hard to take it back So if by the time the bar closes And you feel like falling down I'll carry you home Tonight We are young So let's set the world on fire We can burn brighter than the sun Tonight We are young So let's set the world on fire We can burn brighter than the sun Now I know that I'm not All that you got I guess that I, I just thought

,Song Title,Artist,Male0 / Female1/Other3,Publish Year,Length of Song,Length in Seconds,Solo0 / Group1,Main Chord Progression,Romance,Lyrics,bpm,danceability
1,We Are Young,Fun,3,2012,1,"1,6,4,5,1,6,4,5",0,"Give me a second I, I need to get my story str...",4:12,252,,
2,Save Your Tears,The Weeknd,0,2020,0,"2,6,4,4,1,6,3,5",1,Ooh Yeah I saw you dancing in a crowded room Y...,3:35,215,117.7239685,1.513066888
3,Blinding Lights,The Weeknd,0,2020,0,"2,6,1,5,2,6,1,5",1,Yeah I've been tryin' to call I've been on my ...,4:22,262,85.51095581,1.261864305
4,Rain on me,"Lady Gaga, Ariana Grande",1,2020,0,"1,5,6,5,1,6,5,1",0,I didn't ask for a free ride I only asked you ...,3:08,188,123.0911636,1.185065985
5,Intentions,Justin Bieber,0,2020,0,"1,1,6,4,6,4,1,1",1,"Picture perfect, you don't need no filter Gorg...",3:44,224,147.9904785,1.051465869
...,...,...,...,...,...,...,...,...,...,...,...,...
197,Rumor Has It,Adele,1,2012,0,"1,5,6,3,1,5,6,3",1,"Ooh, ooh, ooh, ooh She, she ain't real She ain...",,,,
198,Fly Me To The Moon,Frank Sinatra,0,1954,0,"1,3,6,2,5,3,4,1",1,Fly me to the moon Let me play among the stars...,,,117.3083038,0.9752522707
199,Bad Romance,Lady Gaga,1,2009,0,"1,2,5,1,2,6,1,2",1,Oh-oh-oh-oh-oh-oh-oh-oh-oh-oh-oh-oh! Caught in...,,,118.2536697,1.294276595
200,Turning Tables,Adele,1,2015,0,"1,6,3,6,1,6,3,6",1,Close enough to start a war All that I have is...,,,,


In [ ]:
english_bpm = english.iloc[:,[0,10,11]]
english_bpm
english_bpm['bpm'].replace('', np.nan, inplace=True)
english_bpm_clean = english_bpm.dropna(subset = ["bpm"])
english_bpm_clean

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,Song Title,bpm,danceability
2,Save Your Tears,117.7239685,1.513066888
3,Blinding Lights,85.51095581,1.261864305
4,Rain on me,123.0911636,1.185065985
5,Intentions,147.9904785,1.051465869
6,Mood,90.81459808,1.072853923
...,...,...,...
195,Indigo,144.7736206,1.237753153
196,bury a friend,118.3728333,1.066213965
198,Fly Me To The Moon,117.3083038,0.9752522707
199,Bad Romance,118.2536697,1.294276595


In [ ]:
eng_X = english_bpm_clean.iloc[:, 1:3].values
eng_y = english_bpm_clean.iloc[:, 0].values

### Run KNN Classfier

In [ ]:
# Initiate classifier using mandarin data
classifier_mandarin = KNeighborsClassifier(n_neighbors=3)
classifier_mandarin.fit(man_X, man_y)
# Input bpm,dancebility Nx2 array to classifier, then predict neighbors for N rows
predictions_english = classifier_mandarin.predict(eng_X) # input Nx2 array
matches_for_english = pd.DataFrame(predictions_english, eng_y, columns=['Mandarin Song Match'])
matches_for_english

,Mandarin Song Match
Save Your Tears,兜圈
Blinding Lights,情歌
Rain on me,不醉不会
Intentions,国王与乞丐
Mood,在这座城市遗失了你
...,...
Indigo,寻人启事
bury a friend,像风一样
Fly Me To The Moon,像风一样
Bad Romance,像风一样


In [ ]:
# Find distances to neighbor for input [bpm, dancebility] of 1x2 array
distances, indices = classifier_mandarin.kneighbors([eng_X[0]], n_neighbors=3)
distances
mandarin.iloc[indices[0]]

,Song Title,Artist,Male0 / Female1/Other3,Publish Year,Length of Song,Length in Seconds,Solo0 / Group1,Main Chord Progression,Romance,Lyrics,Translated Lyrics,bpm,danceability
90,天真有邪,林宥嘉,0,2016,5:42,342,0,"1,3,7,6,4,4,7,3",1,我已經 不能用單純 的語氣 再唱情歌 雖然表面上 我還是完整 那個我 可是身體𥚃 有個什麼 ...,I can not sing love songs with simple tone on ...,128.9712219,0.9254204631
27,歌之王,陈奕迅,0,2013,3:51,231.00,0,"1,5,6,3,4,2,4,5",1,我唱得不够动人 你别皱眉 我愿意和你 约定至死 我只想嬉戏唱游 到下世纪 请你别嫌我将这煽情...,I do not sing enough touching you and you frow...,153.3956299,0.9789291024
210,可惜不是你,梁静茹,1,2012,4:48,288,0,"4,5,3,6,4,5,1,4",1,这一刻突然觉得好熟悉像昨天今天同时在放映我这句语气原来好像你不就是我们爱过的证据差一点骗了自...,"At this moment I suddenly felt so familiar, li...",,


In [ ]:
# Initiate classifier
classifier_english = KNeighborsClassifier(n_neighbors=3)
classifier_english.fit(eng_X, eng_y)

# Input bpm,dancebility 2x1 array to classifier, then predict neighbors
predictions_english = classifier_english.predict(man_X)
matches_for_mandarin = pd.DataFrame(predictions_english, man_y, columns=['Mandarin Song Match'])
matches_for_mandarin

,Mandarin Song Match
我很快乐,Like I'm Gonna Lose You
过,Look What You Made Me Do
很久以后,Arms
莲,How Long
好想爱这个世界啊,Big Girls Cry
...,...
水调歌头,Baby
淒美地,California Gurls
你是如此难以忘记,Fly Me To The Moon
致姍姍來遲的你,Maps


In [ ]:
distances, indices = classifier_english.kneighbors([eng_X[0]], n_neighbors=4)
distances, indices

(array([[0.        , 0.03732703, 0.54653567, 0.5731078 ]]),
 array([[  0,   8,  92, 154]]))